In [ ]:
!pip install pandarallel
import pandas as pd
pd.set_option("max_colwidth", 600)
from bs4 import BeautifulSoup
import requests

from pandarallel import pandarallel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def extract_urls_topic(url):
    try:
      coverpage = requests.get(url).content
      soup = BeautifulSoup(coverpage, 'html.parser')
     
      rows = []
      coverpage=soup.find_all('loc')
      for link in coverpage:
        if link.get_text().find('https://www.laopinioncoruna.es/mundo')>=0 :
          rows.append(link.get_text()) 
        else :
          if link.get_text().find('https://www.laopinioncoruna.es/espana')>=0:
            rows.append(link.get_text()) 
          else :
            if link.get_text().find('https://www.laopinioncoruna.es/local')>=0:
              rows.append(link.get_text()) 
    except:
      rows = 'error'
      
    return rows

In [ ]:
extract_urls_topic('https://www.laopinioncoruna.es/sitemap_contents_2022_05_967c1_001.xml')

['https://www.laopinioncoruna.es/espana/2022/06/01/fiscal-general-vuelve-baja-presidir-66762573.html',
 'https://www.laopinioncoruna.es/mundo/2022/05/31/rusia-cortara-miercoles-gas-dinamarca-66750600.html',
 'https://www.laopinioncoruna.es/mundo/2022/05/31/tusk-despide-presidencia-partido-popular-66755506.html',
 'https://www.laopinioncoruna.es/mundo/2022/05/31/muerto-heridos-tiroteo-universidad-nueva-66760539.html',
 'https://www.laopinioncoruna.es/espana/2022/05/31/gobierno-limita-propuesta-descentralizacion-puig-66755150.html',
 'https://www.laopinioncoruna.es/espana/2022/05/31/feijoo-denuncia-von-der-leyen-66758833.html',
 'https://www.laopinioncoruna.es/espana/2022/05/31/sanchez-afirma-abierto-prorrogar-medidas-66746323.html',
 'https://www.laopinioncoruna.es/espana/2022/05/31/ayuso-avanza-plan-revisar-nuevos-66758489.html',
 'https://www.laopinioncoruna.es/espana/2022/05/31/fiscalia-europea-ve-indicios-madi-66756651.html',
 'https://www.laopinioncoruna.es/mundo/2022/05/31/conflic

In [ ]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

url_str=''

try:
  coverpage = requests.get('https://www.laopinioncoruna.es/sitemap_index_967c1.xml').content
  soup = BeautifulSoup(coverpage, 'html.parser')
  
  rows =[]
  coverpage=soup.find_all('loc')
  for link in coverpage:
    rows.append(link.get_text()) 
except:
  rows = 'error'
df = pd.DataFrame(range(0,60),columns = ['page'])
df['page']=rows
print(df)
df['rows']=df['page'].parallel_apply(extract_urls_topic)
df = df[df.rows.str.len()!=0].reset_index(drop=True)
df = df.explode('rows', ignore_index=True)
df=df.drop_duplicates(subset='rows')
df = df.dropna()
print(df)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/laopinioncoruna/url.csv',index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
                                                                        page
0      https://www.laopinioncoruna.es/sitemap_contents_2022_08_967c1_001.xml
1                     https://www.laopinioncoruna.es/sitemap_today_967c1.xml
2               https://www.laopinioncoruna.es/sitemap_google_news_967c1.xml
3             https://www.laopinioncoruna.es/sitemap_community_967c1_001.xml
4                  https://www.laopinioncoruna.es/sitemap_city_967c1_001.xml
5                   https://www.laopinioncoruna.es/sitemap_tag_967c1_002.xml
6                   https://www.laopinioncoruna.es/sitemap_tag_967c1_001.xml
7               https://www.laopinioncoruna.es/sitemap_section_967c1_001.xml
8             https://www.laopinioncoruna.es/sitemap_microsite_967c1_001.xml
9                  https://www.laopinioncoruna.es/sitemap_blog_967c1_001.xml
10        

                                                                        page  \
0      https://www.laopinioncoruna.es/sitemap_contents_2022_08_967c1_001.xml   
1      https://www.laopinioncoruna.es/sitemap_contents_2022_08_967c1_001.xml   
2      https://www.laopinioncoruna.es/sitemap_contents_2022_08_967c1_001.xml   
3      https://www.laopinioncoruna.es/sitemap_contents_2022_08_967c1_001.xml   
4      https://www.laopinioncoruna.es/sitemap_contents_2022_08_967c1_001.xml   
...                                                                      ...   
26630  https://www.laopinioncoruna.es/sitemap_contents_2020_10_967c1_001.xml   
26631  https://www.laopinioncoruna.es/sitemap_contents_2020_10_967c1_001.xml   
26632  https://www.laopinioncoruna.es/sitemap_contents_2020_10_967c1_001.xml   
26633  https://www.laopinioncoruna.es/sitemap_contents_2020_10_967c1_001.xml   
26634  https://www.laopinioncoruna.es/sitemap_contents_2020_10_967c1_001.xml   

                                       